In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# for the model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, RepeatVector, TimeDistributed, Reshape
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

def fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    new_date = date + timedelta(hours=1)
    value_new[40]=new_date
    value_new[date.hour]=0.0
    value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df

def Y_fill_with_average(df, previous, following):
    value_new = list(df.loc[previous])
    value_previous = list(df.loc[previous])
    value_following = list(df.loc[following])
    value_new[0]= (value_previous[0]+value_following[0])/2
    date = df.loc[previous, 'TimeHour2']
    new_date = date + timedelta(hours=1)
    value_new[2]=new_date
    #value_new[date.hour]=0.0
    #value_new[new_date.hour]=1.0
    #value_new[43]=1.0
    df_length = len(df)
    df.loc[df_length] = value_new
    df = df.sort_values(by= "TimeHour2")
    df = df.reset_index()
    df = df.drop(columns=["index"])
    return df



In [5]:
X = np.load('X.npy')
y = np.load('y.npy')
TimeHour = pd.read_csv('TimeHour.csv')

df_test = pd.DataFrame(data=X)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']
df_test['TimeHour2'] = 'NA'
df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
df_test['Diff'] = 'NA'
df_test['Before']="NA"
for x in range(1, df_test.shape[0]):
    before = df_test.loc[x - 1, 'TimeHour2']
    df_test.loc[x, 'Before'] = before
    now = df_test.loc[x, 'TimeHour2']
    diff = now - before
    df_test.loc[x, 'Diff'] = diff
df_test
df_test = df_test.drop(0)
df_test['diff_hours'] = [x.total_seconds() / 3600 for x in df_test['Diff']]



df_test = df_test.drop([1141,1142])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([655,656,657,658,659,660,661,662,663,664,665,666,667,668,669])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([641,642,643,644,645,646,647,648,649,650,651,652,653,654,655])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test = df_test.drop([1480])
df_test = df_test.drop([7900,7901,7902,7903,7904,7905])
df_test = df_test.reset_index()
df_test = df_test.drop(columns=["index"])
df_test[df_test['diff_hours']>1]


,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,TimeHour,TimeHour2,Diff,Before,diff_hours
550,3.715820e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,28.0
662,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,101.0
1037,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,51.0
1110,2.441406e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 13:00:00,0 days 04:00:00,2018-02-28 09:00:00,4.0
1479,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,503.0
1661,9.765621e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-13 16:00:00,2018-04-13 16:00:00,0 days 06:00:00,2018-04-13 10:00:00,6.0
1774,3.749680e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-18 17:00:00,2018-04-18 17:00:00,0 days 09:00:00,2018-04-18 08:00:00,9.0
1793,-4.523088e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,25.0
2133,-7.734648e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-05-04 18:00:00,2018-05-04 18:00:00,0 days 03:00:00,2018-05-04 15:00:00,3.0
3747,0.000000e+00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,2018-07-11 01:00:00,2018-07-11 01:00:00,0 days 02:00:00,2018-07-10 23:00:00,2.0


In [66]:
x = 0
while x < 13000:
    #if df_test.loc[x, 'diff_hours'] != 1.0 and df_test.loc[x, "diff_hours"] != 24.0 and df_test.loc[x, 'diff_hours'] != 96.0 and df_test.loc[x, 'diff_hours'] != 48.0 and df_test.loc[x, 'diff_hours'] != 480.0:
    if df_test.loc[x,'diff_hours'] %24!=0.0 and df_test.loc[x,'diff_hours'] != 1.0:
        print(x)
        df_test.loc[x, 'diff_hours'] = df_test.loc[x, 'diff_hours'] - 1
        df_test = Y_fill_with_average(df_test, x-1, x)
        x = x - 1
    else:
        x = x + 1

print ('Missing values approximated')
df_test[df_test['diff_hours']>1]

550
551
552
553
666
667
668
669
670
1046
1047
1048
1122
1123
1124
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1699
1700
1701
1702
1703
1817
1818
1819
1820
1821
1822
1823
1824
1844
2185
2186
3801
4861
4862
5325
5326
5327
6682
6683
6684
6685
6686
6687
6688
10091
12717
Missing values approximated


,0,TimeHour,TimeHour2,Diff,Before,diff_hours
554,47.628271,2018-01-25 14:00:00,2018-01-25 14:00:00,1 days 04:00:00,2018-01-24 10:00:00,24.0
671,0.000000,2018-02-05 16:00:00,2018-02-05 16:00:00,4 days 05:00:00,2018-02-01 11:00:00,96.0
1049,76.974656,2018-02-23 09:00:00,2018-02-23 09:00:00,2 days 03:00:00,2018-02-21 06:00:00,48.0
1517,35.789604,2018-04-05 20:00:00,2018-04-05 20:00:00,20 days 23:00:00,2018-03-15 21:00:00,480.0
1845,45.334073,2018-04-20 12:00:00,2018-04-20 12:00:00,1 days 01:00:00,2018-04-19 11:00:00,24.0


In [68]:

df_test = df_test.drop(['Diff', 'TimeHour2', 'diff_hours', 'Before'],axis=1)
df_test = df_test.drop([13632,13633,13634,13635,13636,13637,13638,13639,13640])

print(df_test.shape)
df_test

(13632, 2)


,0,TimeHour
0,502.995390,2018-01-01 13:00:00
1,501.561763,2018-01-01 14:00:00
2,267.722037,2018-01-01 15:00:00
3,164.684623,2018-01-01 16:00:00
4,415.606068,2018-01-01 17:00:00
...,...,...
13627,95.319013,2019-08-25 10:00:00
13628,158.127773,2019-08-25 11:00:00
13629,107.449515,2019-08-25 12:00:00
13630,184.320927,2019-08-25 13:00:00


In [69]:
#np.save('X_cleaned.npy', df_test)